In [1]:
import sys
sys.path.append('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github')

In [3]:

# import pysparnn.cluster_index as ci
# import scipy.sparse
# import os
import math
import numpy as np
import pandas as pd
import random
import scipy.sparse as sp
from utils import train_test_split, df_to_matrix ,matrix_to_df_2, threshold_interactions_df, matrix_to_df,set_intersection,get_0_and_p_index,set_diff, matrix_to_full_df, threshold_interactions_df_plus, train_test_split_csr

!pip install surprise
from surprise import Reader, accuracy
from surprise import SVD
from surprise import Dataset
from collections import Counter
from collections import defaultdict

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from pct.tree.heuristic.Heuristic import Heuristic5
from pct.tree.heuristic.NumericHeuristic_yahoo import NumericHeuristic5
from pct.tree.splitter.splitter import Splitter
from pct.tree.tree import Tree


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [27]:

my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)

def load_user_item_matrix(filepath):
    data = []

    with open(filepath, 'r') as f:
        while True:
            user_line = f.readline()
            if not user_line:
                break

            user_line = user_line.strip()
            if '|' not in user_line:
                continue

            user_id, num_ratings = user_line.split('|')
            user_id = int(user_id)
            num_ratings = int(num_ratings)

            for _ in range(num_ratings):
                item_line = f.readline().strip()
                parts = item_line.split('\t')
                if len(parts) < 2:
                    continue
                item_id, score = parts[0], parts[1]
                data.append([user_id, int(item_id), int(score)])

    df = pd.DataFrame(data, columns=['user_id', 'item_id', 'rating'])
    return df


In [28]:
def safe_int(x):
    return int(x) if x != 'None' else 0


def load_track_data(filepath):
    track_data = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('|')
            track_id = safe_int(parts[0])
            album_id = safe_int(parts[1])
            artist_id = safe_int(parts[2])
            genre_ids = [safe_int(g) for g in parts[3:]] if len(parts) > 3 else []
            track_data.append([track_id, album_id, artist_id, genre_ids])
    return pd.DataFrame(track_data, columns=['track_id', 'album_id', 'artist_id', 'genre_ids'])

def load_album_data(filepath):
    album_data = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('|')
            album_id = safe_int(parts[0])
            artist_id = safe_int(parts[1])
            genre_ids = [safe_int(g) for g in parts[2:]] if len(parts) > 2 else []
            album_data.append([album_id, artist_id, genre_ids])
    return pd.DataFrame(album_data, columns=['album_id', 'artist_id', 'genre_ids'])


# read artistData2
def load_artist_data(filepath):
    with open(filepath, 'r') as f:
        artist_ids = [int(line.strip()) for line in f]
    return pd.DataFrame(artist_ids, columns=['artist_id'])

# read genreData2
def load_genre_data(filepath):
    with open(filepath, 'r') as f:
        genre_ids = [int(line.strip()) for line in f]
    return pd.DataFrame(genre_ids, columns=['genre_id'])

In [29]:
track_df = load_track_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/trackData2.txt')
track_df

,track_id,album_id,artist_id,genre_ids
0,1,106710,281667,"[214765, 162234, 155788]"
1,2,280977,233685,"[131552, 173467, 48505]"
2,3,38422,219136,"[61215, 201738, 88853]"
3,4,119529,166863,"[17453, 35389]"
4,5,16742,294690,"[61215, 34486, 274088]"
...,...,...,...,...
224036,296100,166516,33011,"[274088, 199606, 88853]"
224037,296101,0,0,[]
224038,296102,153644,289056,"[158282, 139095, 242383]"
224039,296105,68336,6613,[82064]


In [30]:
album_df = load_album_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/albumData2.txt')
album_df

,album_id,artist_id,genre_ids
0,0,0,[214765]
1,6,228091,"[158282, 81520, 242383]"
2,19,85028,[103715]
3,30,16832,[31567]
4,33,26330,"[149962, 209270]"
...,...,...,...
52824,296104,125866,[158282]
52825,296106,221637,"[116130, 9432]"
52826,296107,0,[61215]
52827,296108,93277,[61215]


In [31]:
artist_df = load_artist_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/artistData2.txt')
artist_df

,artist_id
0,16
1,23
2,35
3,40
4,49
...,...
18669,295987
18670,296007
18671,296012
18672,296013


In [32]:
genre_df = load_genre_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/genreData2.txt')
genre_df

,genre_id
0,208
1,315
2,642
3,1075
4,1271
...,...
562,292093
563,293670
564,293688
565,294138


In [33]:
train_path = '/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/trainIdx2.txt'
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)
train_df = load_user_item_matrix(train_path)


In [10]:
train_df

,user_id,item_id,rating
0,0,28341,90
1,0,51210,90
2,0,79500,90
3,0,82317,90
4,0,98399,90
...,...,...,...
61944401,249011,270557,90
61944402,249011,273574,90
61944403,249011,286938,90
61944404,249011,287681,80


In [34]:
duplicate_counts = train_df.groupby(['user_id', 'item_id']).size()
duplicates = duplicate_counts[duplicate_counts > 1]

print(f"🔁 Number of duplicated (user_id, item_id) pairs: {len(duplicates)}")
print(duplicates.head())


🔁 Number of duplicated (user_id, item_id) pairs: 0
Series([], dtype: int64)


In [12]:
print(train_df['item_id'].max())      # highest item_id
print(track_df['track_id'].max())     # highest track_id
print(train_df['item_id'].isin(track_df['track_id']).mean())  # proportion to match

print(train_df['item_id'].isin(album_df['album_id']).mean())

print(train_df['item_id'].isin(artist_df['artist_id']).mean())

print(train_df['item_id'].isin(genre_df['genre_id']).mean())



296110
296110
0.4385845107627636
0.1925648621120041
0.3114063600835885
0.05744426704164376


In [35]:
def build_item_metadata(track_df, album_df, artist_df, genre_df):
    rows = []

    # 1. Tracks
    for _, row in track_df.iterrows():
        rows.append({
            'item_id': row['track_id'],
            'item_type': 'track',
            'artist_id': row['artist_id'],
            'genre_ids': row['genre_ids']
        })

    # 2. Albums
    for _, row in album_df.iterrows():
        rows.append({
            'item_id': row['album_id'],
            'item_type': 'album',
            'artist_id': row['artist_id'],
            'genre_ids': row['genre_ids']
        })

    # 3. Artists
    for artist_id in artist_df['artist_id']:
        rows.append({
            'item_id': artist_id,
            'item_type': 'artist',
            'artist_id': artist_id,
            'genre_ids': 0
        })

    # 4. Genres
    for genre_id in genre_df['genre_id']:
        rows.append({
            'item_id': genre_id,
            'item_type': 'genre',
            'artist_id': 0,
            'genre_ids': [genre_id]
        })

    return pd.DataFrame(rows)

In [36]:
item_meta_df = build_item_metadata(track_df, album_df, artist_df, genre_df)

train_with_meta = train_df.merge(item_meta_df, on='item_id', how='inner')

# When loading the data, sort by a stable column (e.g., user_id)
train_with_meta = train_with_meta.sort_values('user_id').reset_index(drop=True)

train_with_meta

,user_id,item_id,rating,item_type,artist_id,genre_ids
0,0,28341,90,artist,28341,0
1,0,232604,30,artist,232604,0
2,0,237425,50,artist,237425,0
3,0,245257,50,artist,245257,0
4,0,23301,50,artist,23301,0
...,...,...,...,...,...,...
61944401,249011,141075,90,artist,141075,0
61944402,249011,150385,80,genre,0,[150385]
61944403,249011,152077,90,artist,152077,0
61944404,249011,158282,80,genre,0,[158282]


In [ ]:
df = train_with_meta.copy()

# Filter: keep only artist & genre (drop album, track)
df = df[~df['item_type'].isin(['album', 'track'])].copy()

print(f"🎯 Sample shape: {df.shape}")
print(f"👤 Users: {df['user_id'].nunique()}, 🎵 Items: {df['item_id'].nunique()}")

🎯 Sample shape: (22848233, 6)
👤 Users: 248344, 🎵 Items: 19241


In [38]:
filtered_df = threshold_interactions_df(df,'user_id','item_id',100,500)

Starting interactions info
Number of rows: 248344
Number of cols: 19241
Sparsity: 0.478%
Ending interactions info
Number of rows: 4644
Number of columns: 8938
Sparsity: 10.674%


In [39]:
# filtered_df = threshold_interactions_df_plus(df, min_items_per_user=500, min_artists_per_user=20, min_genres_per_user=10,min_users_per_item=150)
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)
sampled_users = np.random.choice(filtered_df['user_id'].unique(), size=2000, replace=False)
sampled_df = filtered_df[filtered_df['user_id'].isin(sampled_users)]
print(f"🎯 Sample shape: {sampled_df.shape}")
print(f"👤 Users: {sampled_df['user_id'].nunique()}, 🎵 Items: {sampled_df['item_id'].nunique()}")
filtered_df = threshold_interactions_df(sampled_df,'user_id','item_id',100,500)
print(f"🎯 Sample shape: {filtered_df.shape}")

🎯 Sample shape: (1941009, 6)
👤 Users: 2000, 🎵 Items: 8938
Starting interactions info
Number of rows: 2000
Number of cols: 8938
Sparsity: 10.858%
Ending interactions info
Number of rows: 1701
Number of columns: 5011
Sparsity: 17.890%
🎯 Sample shape: (1524894, 6)


Sample shape: (1941009, 6)
👤 Users: 2000, 🎵 Items: 8938
Starting interactions info
Number of rows: 2000
Number of cols: 8938
Sparsity: 10.858%
Ending interactions info
Number of rows: 1701
Number of columns: 5011
Sparsity: 17.890%

In [42]:
filtered_df

,user_id,item_id,rating,item_type,artist_id,genre_ids
1039,9,182744,30,artist,182744,0
1040,9,197011,50,artist,197011,0
1041,9,160820,0,artist,160820,0
1043,9,59402,30,artist,59402,0
1044,9,251901,0,artist,251901,0
...,...,...,...,...,...,...
61933948,248947,210727,50,artist,210727,0
61933956,248947,275456,0,artist,275456,0
61933959,248947,203741,0,artist,203741,0
61933965,248947,202602,0,artist,202602,0


In [90]:
# export filtered_df 
filtered_df.to_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/re_sample/filtered_df.csv', index=False)
# Load the filtered DataFrame
filtered_df = pd.read_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/re_sample/filtered_df.csv')

In [4]:
filtered_df = pd.read_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/re_sample/filtered_df.csv')

In [6]:


# Number of unique users
num_users = filtered_df['user_id'].nunique()
print("Number of unique users:", num_users)

# Number of unique artists (item_type == 'artist')
num_artists = filtered_df[filtered_df['item_type'] == 'artist']['item_id'].nunique()
print("Number of unique artists:", num_artists)

# Number of unique genres (item_type == 'genre')
num_genres = filtered_df[filtered_df['item_type'] == 'genre']['item_id'].nunique()
print("Number of unique genres:", num_genres)


Number of unique users: 1701
Number of unique artists: 4793
Number of unique genres: 218


In [5]:
filtered_df

,user_id,item_id,rating,item_type,artist_id,genre_ids
0,9,182744,30,artist,182744,0
1,9,197011,50,artist,197011,0
2,9,160820,0,artist,160820,0
3,9,59402,30,artist,59402,0
4,9,251901,0,artist,251901,0
...,...,...,...,...,...,...
1524889,248947,210727,50,artist,210727,0
1524890,248947,275456,0,artist,275456,0
1524891,248947,203741,0,artist,203741,0
1524892,248947,202602,0,artist,202602,0
